In [46]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
#from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
import math
import numpy as np
from sklearn.model_selection import train_test_split


# DATASET


In [47]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/valid/'
TRAIN_SAMPLES = 2000
VALIDATION_SAMPLES = 360
NUM_CLASSES = 3
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [48]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [49]:
train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=12345,
                        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                        VALIDATION_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        class_mode='categorical')


Found 10604 images belonging to 3 classes.
Found 1848 images belonging to 3 classes.


In [50]:
def model_maker():
    base_model = ResNet50(include_top=False, input_shape =
                    (IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)


In [51]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer= tensorflow.keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)
model.fit_generator(train_generator,
                    steps_per_epoch = num_steps,
                    epochs=30,
                    validation_data = validation_generator,
                    validation_steps = num_steps)

C:\Users\pendo\AppData\Local\Temp\ipykernel_23536\3648776649.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/30
32/32 [==============================] - 40s 1s/step - loss: 0.8200 - acc: 0.6704 - val_loss: 0.8423 - val_acc: 0.6483
Epoch 2/30
32/32 [==============================] - 26s 802ms/step - loss: 0.6443 - acc: 0.7559
Epoch 3/30
32/32 [==============================] - 24s 747ms/step - loss: 0.5975 - acc: 0.7729
Epoch 4/30
32/32 [==============================] - 26s 801ms/step - loss: 0.5588 - acc: 0.7915
Epoch 5/30
32/32 [==============================] - 24s 730ms/step - loss: 0.5355 - acc: 0.8077
Epoch 6/30
32/32 [==============================] - 22s 664ms/step - loss: 0.4950 - acc: 0.8232
Epoch 7/30
32/32 [==============================] - 21s 658ms/step - loss: 0.5298 - acc: 0.8125
Epoch 8/30
32/32 [==============================] - 21s 634ms/step - loss: 0.5073 - acc: 0.8164
Epoch 9/30
32/32 [==============================] - 21s 643ms/step - loss: 0.5301 - acc: 0.8166
Epoch 10/30
32/32 [==============================] - 21s 646ms/step - loss: 0.5044 - acc: 0.8188
Epoch

In [52]:
img_path = 'sample_images/Олень6.jpg'
img = tensorflow.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
img_array = tensorflow.keras.preprocessing.image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array) # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

1/1 [==============================] - 1s 1s/step
[[0.30625647 0.28725535 0.4064882 ]]
{'0': 0, '1': 1, '2': 2}


In [44]:
model.save("model1.h5")